In [ ]:
!pip install ultralytics
!pip install pandas openpyxl

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from ultralytics import YOLO  
import pandas as pd  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 10  
dense_model = models.densenet121(pretrained=False)  
dense_model.classifier = torch.nn.Linear(dense_model.classifier.in_features, num_classes) 
dense_model.load_state_dict(torch.load('/kaggle/input/yolo11_misahub/pytorch/default/8/best_model_dense.pt'))  
dense_model = dense_model.to(device)

yolo_model = YOLO('/kaggle/input/yolo11_misahub/pytorch/default/8/best.pt')  

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  
])

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from ultralytics import YOLO  
import pandas as pd  
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Ensemble Function
def ensemble_prediction(input_image_path):
    image = Image.open(input_image_path).convert("RGB")  
    input_tensor = transform(image).unsqueeze(0).to(device)  #

    dense_model.eval()
    with torch.no_grad():
        dense_outputs = dense_model(input_tensor)
        dense_probs = torch.nn.functional.softmax(dense_outputs, dim=1)

    with torch.no_grad():
        results = yolo_model(input_image_path)  
        
        yolo_probs = results[0].probs.data  

        if yolo_probs.dim() == 1: 
            yolo_probs = yolo_probs.unsqueeze(0)  
        elif yolo_probs.dim() == 2:  
            yolo_probs = yolo_probs[0]  

    
    combined_probs = (dense_probs + yolo_probs) / 2  

    final_class_index = torch.argmax(combined_probs, dim=1).item()
    return dense_probs.squeeze().cpu().numpy(), final_class_index  

def predict_on_folder(folder_path, output_excel_path, class_names):
    results = []

    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  
            image_path = os.path.join(folder_path, filename)
            dense_probs, final_class_index = ensemble_prediction(image_path)
            final_class_name = class_names[final_class_index]  
            results.append([image_path] + dense_probs.tolist() + [final_class_name])

    columns = ['Image Path'] + class_names + ['Predicted Class']
    df = pd.DataFrame(results, columns=columns)
    df.to_excel(output_excel_path, index=False)

val_folder_path = '/kaggle/input/misahub-capsule-2/pif/val'  
class_names = sorted(os.listdir(val_folder_path))  

folder_path = '/kaggle/input/misahub-capsule-2/Testing set/Images'  
output_excel_path = '/kaggle/working/DS & Chill_final.xlsx'  


In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def validate_on_folder(val_folder_path):
    results = []
    class_names = sorted(os.listdir(val_folder_path))  
    num_classes = len(class_names)  

    true_labels = []  
    predicted_classes = []  

    for class_name in class_names:
        class_folder = os.path.join(val_folder_path, class_name)  
        if os.path.isdir(class_folder):  
            for filename in os.listdir(class_folder):
                if filename.endswith(('.png', '.jpg', '.jpeg')):  
                    image_path = os.path.join(class_folder, filename)

                    true_label = class_names.index(class_name)  
                    true_labels.append(true_label)

                    dense_probs, final_class = ensemble_prediction(image_path)
                    predicted_classes.append(final_class)

                    results.append([image_path] + dense_probs.tolist() + [final_class])

    overall_accuracy = accuracy_score(true_labels, predicted_classes)
    precision = precision_score(true_labels, predicted_classes, average='weighted')
    recall = recall_score(true_labels, predicted_classes, average='weighted')
    f1 = f1_score(true_labels, predicted_classes, average='weighted')

    class_accuracies = {}
    for i, class_name in enumerate(class_names):
        class_true_labels = [1 if label == i else 0 for label in true_labels]
        class_predicted_labels = [1 if pred == i else 0 for pred in predicted_classes]
        
        class_accuracy = accuracy_score(class_true_labels, class_predicted_labels)
        class_accuracies[class_name] = class_accuracy

    print("Overall Metrics:")
    print(f'Overall Accuracy: {overall_accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}\n')

    print("Class Accuracies:")
    for class_name, accuracy in class_accuracies.items():
        print(f'{class_name}: {accuracy:.4f}')

val_folder_path = '/kaggle/input/misahub-capsule-2/given_val/given_val'  # Update with your validation folder path
validate_on_folder(val_folder_path)



image 1/1 /kaggle/input/misahub-capsule-2/given_val/given_val/Normal/3c8d5f0b90d7475d_11353.jpg: 224x224 Normal 0.90, Angioectasia 0.06, Lymphangiectasia 0.02, Erosion 0.01, Foreign Body 0.01, 9.5ms
Speed: 1.8ms preprocess, 9.5ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /kaggle/input/misahub-capsule-2/given_val/given_val/Normal/5e59c7fdb16c4228_8157.jpg: 224x224 Normal 1.00, Erosion 0.00, Lymphangiectasia 0.00, Angioectasia 0.00, Bleeding 0.00, 8.9ms
Speed: 1.8ms preprocess, 8.9ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /kaggle/input/misahub-capsule-2/given_val/given_val/Normal/image01488.jpg: 224x224 Lymphangiectasia 0.44, Polyp 0.27, Angioectasia 0.17, Erosion 0.10, Bleeding 0.01, 9.0ms
Speed: 1.8ms preprocess, 9.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /kaggle/input/misahub-capsule-2/given_val/given_val/Normal/495f16498db34d3c_10849.jpg: 224x224 Normal 1.00, Ulcer 0.00, Ero